In [1]:
import pandas as pd
import json
import requests
import math
import numpy as np
import urllib

In [2]:
import utilcalcs as calc
import geo_agg as geo
from censusAPI import myAPI
from county_codes import stco

In [3]:
date = '211004'

## Create a table of all U.S. counties with Labor Force by Age & MOEs
### 2015-2019 5-Year Average

In [4]:
#My search parameters
y1 = '2019'
y0 = '2010'

cols_b = f'group(B23001)'
cols_d = f'GEO_ID,DP03_0002E,DP03_0002M'

bsource = 'acs/acs5'
dsource = 'acs/acs5/profile'

In [5]:
def get_data(year,source,cols):
    url = f"https://api.census.gov/data/{year}/{source}?get={cols}&for=county:*&in=state:*&key={myAPI}"
    resp = requests.request('GET', url).content
    df_co = pd.DataFrame(json.loads(resp)[1:])
    df_co.columns = json.loads(resp)[0]

    url = f"https://api.census.gov/data/{year}/{source}?get={cols}&for=us:*&key={myAPI}"
    resp = requests.request('GET', url).content
    df_us = pd.DataFrame(json.loads(resp)[1:])
    df_us.columns = json.loads(resp)[0]

    df = pd.concat([df_co,df_us],sort=True)
    return df

def clean_data(df,var):
    dff = df[var].copy()
    var_num = var[1:]
    for col in var_num:
        dff[col] = dff[col].astype(float)
    dff = dff.replace([999999999, 555555555, 333333333, 222222222,\
                    666666666, 888888888, -999999999, -555555555,\
                    -333333333, -222222222, -666666666, -888888888], 0)
    return dff

In [6]:
#Grouping variables/columns into lists to run calculations for a new table
#Population and LFor Force Participants by Different Age Cohorts
#Can be used for all ACS years

#Age 16 to 24 
Pop1624E = ['B23001_003E','B23001_010E','B23001_017E','B23001_089E','B23001_096E','B23001_103E']
Pop1624M = ['B23001_003M','B23001_010M','B23001_017M','B23001_089M','B23001_096M','B23001_103M']
LF1624E = ['B23001_004E','B23001_011E','B23001_018E','B23001_090E','B23001_097E','B23001_104E']
LF1624M = ['B23001_004M','B23001_011M','B23001_018M','B23001_090M','B23001_097M','B23001_104M']

#Age 25 to 34
Pop2534E = ['B23001_024E','B23001_031E','B23001_110E','B23001_117E']
Pop2534M = ['B23001_024M','B23001_031M','B23001_110M','B23001_117M']
LF2534E = ['B23001_025E','B23001_032E','B23001_111E','B23001_118E']
LF2534M = ['B23001_025M','B23001_032M','B23001_111M','B23001_118M']

#Age 35 to 44
Pop3544E = ['B23001_038E','B23001_124E']
Pop3544M = ['B23001_038M','B23001_124M']
LF3544E = ['B23001_039E','B23001_125E']
LF3544M = ['B23001_039M','B23001_125M']

#Age 45 to 54
Pop4554E = ['B23001_045E','B23001_131E']
Pop4554M = ['B23001_045M','B23001_131M']
LF4554E = ['B23001_046E','B23001_132E']
LF4554M = ['B23001_046M','B23001_132M']

#Age 25 to 54 (prime-age workforce)
Pop2554E = Pop2534E + Pop3544E + Pop4554E
Pop2554M = Pop2534M + Pop3544M + Pop4554M
LF2554E = LF2534E + LF3544E + LF4554E
LF2554M = LF2534M + LF3544M + LF4554M

#Age 55 to 64
Pop5564E = ['B23001_052E','B23001_059E','B23001_066E','B23001_138E','B23001_145E','B23001_152E']
Pop5564M = ['B23001_052M','B23001_059M','B23001_066M','B23001_138M','B23001_145M','B23001_152M']
LF5564E = ['B23001_053E','B23001_060E','B23001_067E','B23001_139E','B23001_146E','B23001_153E']
LF5564M = ['B23001_053M','B23001_060M','B23001_067M','B23001_139M','B23001_146M','B23001_153M']

#Over Age 65
PopO65E = ['B23001_073E','B23001_078E','B23001_083E','B23001_159E','B23001_164E','B23001_169E']
PopO65M = ['B23001_073M','B23001_078M','B23001_083M','B23001_159M','B23001_164M','B23001_169M']
LFO65E = ['B23001_074E','B23001_079E','B23001_084E','B23001_160E','B23001_165E','B23001_170E']
LFO65M = ['B23001_074M','B23001_079M','B23001_084M','B23001_160M','B23001_165M','B23001_170M']

#Age 55 and over
PopO55E = Pop5564E + PopO65E
PopO55M = Pop5564M + PopO65M
LFO55E = LF5564E + LFO65E
LFO55M = LF5564M + LFO65M

#Total LFor Force
LFE = LF1624E + LF2554E + LFO55E
LFM = LF1624M + LF2554M + LFO55M

#Total Pop
PopE = Pop1624E + Pop2554E + PopO55E
PopM = Pop1624M + Pop2554M + PopO55M

#List of all variables used for calculation + total population variables for spot checking aggregation
var_data = ['GEO_ID','B23001_001E','B23001_001M','DP03_0002E','DP03_0002M'] \
            + PopE + LFE + PopM + LFM

In [7]:
#Pull the data from the variable and profile tables, merge, and clean
dfY1_b = get_data(y1,bsource,cols_b)
dfY1_d = get_data(y1,dsource,cols_d)
dfY1 = pd.merge(dfY1_d,dfY1_b,how='left',on='GEO_ID')
dfY1 = clean_data(dfY1,var_data)
dfY1.head()

,GEO_ID,B23001_001E,B23001_001M,DP03_0002E,DP03_0002M,B23001_003E,B23001_010E,B23001_017E,B23001_089E,B23001_096E,...,B23001_067M,B23001_139M,B23001_146M,B23001_153M,B23001_074M,B23001_079M,B23001_084M,B23001_160M,B23001_165M,B23001_170M
0,0500000US17051,17630.0,131.0,9285.0,356.0,522.0,382.0,438.0,428.0,230.0,...,48.0,97.0,46.0,74.0,61.0,61.0,27.0,54.0,27.0,19.0
1,0500000US17107,24117.0,83.0,13168.0,383.0,830.0,408.0,588.0,855.0,374.0,...,60.0,101.0,51.0,57.0,51.0,66.0,24.0,56.0,31.0,47.0
2,0500000US17165,19422.0,92.0,10718.0,362.0,636.0,302.0,355.0,442.0,284.0,...,75.0,91.0,65.0,68.0,59.0,58.0,29.0,72.0,50.0,45.0
3,0500000US17097,551803.0,565.0,380960.0,2052.0,23800.0,11398.0,16055.0,19905.0,8382.0,...,455.0,668.0,449.0,460.0,414.0,359.0,228.0,435.0,244.0,176.0
4,0500000US17127,11427.0,107.0,5895.0,294.0,363.0,128.0,251.0,293.0,119.0,...,44.0,78.0,49.0,35.0,49.0,19.0,33.0,28.0,32.0,19.0


In [8]:
#Calculate all of the new aggregations

dfY1['Pop_Y1E'] = dfY1['B23001_001E']
dfY1['Pop_Y1M'] = dfY1['B23001_001M']
dfY1['Pop_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Pop_Y1E'],x['Pop_Y1M'])),axis=1)
dfY1['LF_Y1E'] = dfY1['DP03_0002E']
dfY1['LF_Y1M'] = dfY1['DP03_0002M']
dfY1['LF_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF_Y1E'],x['LF_Y1M'])),axis=1)

dfY1['Pop1624_Y1E'] = dfY1.loc[:,Pop1624E].sum(axis=1)
dfY1['Pop1624_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Pop1624M])),axis=1)
dfY1['Pop1624_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Pop1624_Y1E'],x['Pop1624_Y1M'])),axis=1)
dfY1['LF1624_Y1E'] = dfY1.loc[:,LF1624E].sum(axis=1)
dfY1['LF1624_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LF1624M])),axis=1)
dfY1['LF1624_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF1624_Y1E'],x['LF1624_Y1M'])),axis=1)

dfY1['Pop2554_Y1E'] = dfY1.loc[:,Pop2554E].sum(axis=1)
dfY1['Pop2554_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Pop2554M])),axis=1)
dfY1['Pop2554_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Pop2554_Y1E'],x['Pop2554_Y1M'])),axis=1)
dfY1['LF2554_Y1E'] = dfY1.loc[:,LF2554E].sum(axis=1)
dfY1['LF2554_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LF2554M])),axis=1)
dfY1['LF2554_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF2554_Y1E'],x['LF2554_Y1M'])),axis=1)

dfY1['Pop5564_Y1E'] = dfY1.loc[:,Pop5564E].sum(axis=1)
dfY1['Pop5564_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Pop5564M])),axis=1)
dfY1['Pop5564_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Pop5564_Y1E'],x['Pop5564_Y1M'])),axis=1)
dfY1['LF5564_Y1E'] = dfY1.loc[:,LF5564E].sum(axis=1)
dfY1['LF5564_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LF5564M])),axis=1)
dfY1['LF5564_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF5564_Y1E'],x['LF5564_Y1M'])),axis=1)

dfY1['PopO65_Y1E'] = dfY1.loc[:,PopO65E].sum(axis=1)
dfY1['PopO65_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[PopO65M])),axis=1)
dfY1['PopO65_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['PopO65_Y1E'],x['PopO65_Y1M'])),axis=1)
dfY1['LFO65_Y1E'] = dfY1.loc[:,LFO65E].sum(axis=1)
dfY1['LFO65_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LFO65M])),axis=1)
dfY1['LFO65_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LFO65_Y1E'],x['LFO65_Y1M'])),axis=1)

dfY1['Pop2534_Y1E'] = dfY1.loc[:,Pop2534E].sum(axis=1)
dfY1['Pop2534_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Pop2534M])),axis=1)
dfY1['Pop2534_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Pop2534_Y1E'],x['Pop2534_Y1M'])),axis=1)
dfY1['LF2534_Y1E'] = dfY1.loc[:,LF2534E].sum(axis=1)
dfY1['LF2534_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LF2534M])),axis=1)
dfY1['LF2534_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF2534_Y1E'],x['LF2534_Y1M'])),axis=1)

dfY1['Pop3544_Y1E'] = dfY1.loc[:,Pop3544E].sum(axis=1)
dfY1['Pop3544_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Pop3544M])),axis=1)
dfY1['Pop3544_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Pop3544_Y1E'],x['Pop3544_Y1M'])),axis=1)
dfY1['LF3544_Y1E'] = dfY1.loc[:,LF3544E].sum(axis=1)
dfY1['LF3544_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LF3544M])),axis=1)
dfY1['LF3544_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF3544_Y1E'],x['LF3544_Y1M'])),axis=1)

dfY1['Pop4554_Y1E'] = dfY1.loc[:,Pop4554E].sum(axis=1)
dfY1['Pop4554_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[Pop4554M])),axis=1)
dfY1['Pop4554_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['Pop4554_Y1E'],x['Pop4554_Y1M'])),axis=1)
dfY1['LF4554_Y1E'] = dfY1.loc[:,LF4554E].sum(axis=1)
dfY1['LF4554_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LF4554M])),axis=1)
dfY1['LF4554_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LF4554_Y1E'],x['LF4554_Y1M'])),axis=1)

dfY1['PopO55_Y1E'] = dfY1.loc[:,PopO55E].sum(axis=1)
dfY1['PopO55_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[PopO55M])),axis=1)
dfY1['PopO55_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['PopO55_Y1E'],x['PopO55_Y1M'])),axis=1)
dfY1['LFO55_Y1E'] = dfY1.loc[:,LFO55E].sum(axis=1)
dfY1['LFO55_Y1M'] = dfY1.apply(lambda x: (calc.get_moe(x[LFO55M])),axis=1)
dfY1['LFO55_Y1C'] = dfY1.apply(lambda x: (calc.get_cv(x['LFO55_Y1E'],x['LFO55_Y1M'])),axis=1)
    
dfY1.head()

,GEO_ID,B23001_001E,B23001_001M,DP03_0002E,DP03_0002M,B23001_003E,B23001_010E,B23001_017E,B23001_089E,B23001_096E,...,Pop4554_Y1C,LF4554_Y1E,LF4554_Y1M,LF4554_Y1C,PopO55_Y1E,PopO55_Y1M,PopO55_Y1C,LFO55_Y1E,LFO55_Y1M,LFO55_Y1C
0,0500000US17051,17630.0,131.0,9285.0,356.0,522.0,382.0,438.0,428.0,230.0,...,1.436104,2041.0,131.590273,3.919358,7122.0,282.646422,2.412546,2237.0,203.536238,5.531079
1,0500000US17107,24117.0,83.0,13168.0,383.0,830.0,408.0,588.0,855.0,374.0,...,1.926062,2615.0,113.899078,2.647784,9230.0,270.874510,1.784023,3305.0,210.841172,3.878092
2,0500000US17165,19422.0,92.0,10718.0,362.0,636.0,302.0,355.0,442.0,284.0,...,1.254612,2163.0,182.433550,5.127224,8216.0,328.753707,2.432452,2379.0,239.215384,6.112639
3,0500000US17097,551803.0,565.0,380960.0,2052.0,23800.0,11398.0,16055.0,19905.0,8382.0,...,0.039108,86335.0,686.990539,0.483724,190852.0,1753.133195,0.558409,92516.0,1532.247043,1.006807
4,0500000US17127,11427.0,107.0,5895.0,294.0,363.0,128.0,251.0,293.0,119.0,...,1.884413,1321.0,132.136293,6.080698,4988.0,254.287239,3.099076,1136.0,162.806634,8.712201


In [9]:
dfY1 = dfY1.drop(var_data[1:],axis=1)
dfY1.head()

,GEO_ID,Pop_Y1E,Pop_Y1M,Pop_Y1C,LF_Y1E,LF_Y1M,LF_Y1C,Pop1624_Y1E,Pop1624_Y1M,Pop1624_Y1C,...,Pop4554_Y1C,LF4554_Y1E,LF4554_Y1M,LF4554_Y1C,PopO55_Y1E,PopO55_Y1M,PopO55_Y1C,LFO55_Y1E,LFO55_Y1M,LFO55_Y1C
0,0500000US17051,17630.0,131.0,0.451703,9285.0,356.0,2.330785,2327.0,187.728527,4.904198,...,1.436104,2041.0,131.590273,3.919358,7122.0,282.646422,2.412546,2237.0,203.536238,5.531079
1,0500000US17107,24117.0,83.0,0.209213,13168.0,383.0,1.768125,3744.0,220.492630,3.580077,...,1.926062,2615.0,113.899078,2.647784,9230.0,270.874510,1.784023,3305.0,210.841172,3.878092
2,0500000US17165,19422.0,92.0,0.287957,10718.0,362.0,2.053189,2309.0,191.856717,5.051114,...,1.254612,2163.0,182.433550,5.127224,8216.0,328.753707,2.432452,2379.0,239.215384,6.112639
3,0500000US17097,551803.0,565.0,0.062244,380960.0,2052.0,0.327440,94127.0,1381.973589,0.892523,...,0.039108,86335.0,686.990539,0.483724,190852.0,1753.133195,0.558409,92516.0,1532.247043,1.006807
4,0500000US17127,11427.0,107.0,0.569227,5895.0,294.0,3.031780,1406.0,183.267564,7.923816,...,1.884413,1321.0,132.136293,6.080698,4988.0,254.287239,3.099076,1136.0,162.806634,8.712201


dfY1.to_excel('lf18_county_FullUS.xlsx')

## Create a table of all U.S. counties with Labor Force by Age & MOEs
### 2006 - 2010 5-Year Average


In [10]:
dfY0_b = get_data(y0,bsource,cols_b)
dfY0_d = get_data(y0,dsource,cols_d)
dfY0 = pd.merge(dfY0_d,dfY0_b,how='left',on='GEO_ID')
dfY0 = clean_data(dfY0,var_data)
dfY0.head()

,GEO_ID,B23001_001E,B23001_001M,DP03_0002E,DP03_0002M,B23001_003E,B23001_010E,B23001_017E,B23001_089E,B23001_096E,...,B23001_067M,B23001_139M,B23001_146M,B23001_153M,B23001_074M,B23001_079M,B23001_084M,B23001_160M,B23001_165M,B23001_170M
0,0500000US13155,7589.0,99.0,3908.0,649.0,302.0,113.0,194.0,343.0,111.0,...,76.0,90.0,42.0,42.0,52.0,85.0,132.0,22.0,43.0,132.0
1,0500000US13157,44688.0,238.0,28292.0,796.0,1982.0,613.0,1040.0,1439.0,444.0,...,75.0,166.0,128.0,123.0,114.0,64.0,66.0,100.0,52.0,26.0
2,0500000US13159,10609.0,53.0,6797.0,286.0,415.0,126.0,308.0,370.0,167.0,...,56.0,116.0,72.0,50.0,35.0,48.0,43.0,16.0,33.0,40.0
3,0500000US13161,10969.0,153.0,6257.0,369.0,319.0,63.0,310.0,445.0,260.0,...,56.0,101.0,103.0,132.0,65.0,19.0,13.0,46.0,132.0,132.0
4,0500000US13163,13079.0,117.0,7263.0,437.0,596.0,301.0,249.0,514.0,267.0,...,55.0,103.0,37.0,55.0,46.0,38.0,31.0,59.0,17.0,12.0


In [11]:
#Calculate all of the new aggregations

dfY0['Pop_Y0E'] = dfY0['B23001_001E']
dfY0['Pop_Y0M'] = dfY0['B23001_001M']
dfY0['Pop_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['Pop_Y0E'],x['Pop_Y0M'])),axis=1)
dfY0['LF_Y0E'] = dfY0['DP03_0002E']
dfY0['LF_Y0M'] = dfY0['DP03_0002M']
dfY0['LF_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF_Y0E'],x['LF_Y0M'])),axis=1)

dfY0['Pop1624_Y0E'] = dfY0.loc[:,Pop1624E].sum(axis=1)
dfY0['Pop1624_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[Pop1624M])),axis=1)
dfY0['Pop1624_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['Pop1624_Y0E'],x['Pop1624_Y0M'])),axis=1)
dfY0['LF1624_Y0E'] = dfY0.loc[:,LF1624E].sum(axis=1)
dfY0['LF1624_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LF1624M])),axis=1)
dfY0['LF1624_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF1624_Y0E'],x['LF1624_Y0M'])),axis=1)

dfY0['Pop2554_Y0E'] = dfY0.loc[:,Pop2554E].sum(axis=1)
dfY0['Pop2554_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[Pop2554M])),axis=1)
dfY0['Pop2554_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['Pop2554_Y0E'],x['Pop2554_Y0M'])),axis=1)
dfY0['LF2554_Y0E'] = dfY0.loc[:,LF2554E].sum(axis=1)
dfY0['LF2554_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LF2554M])),axis=1)
dfY0['LF2554_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF2554_Y0E'],x['LF2554_Y0M'])),axis=1)

dfY0['Pop5564_Y0E'] = dfY0.loc[:,Pop5564E].sum(axis=1)
dfY0['Pop5564_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[Pop5564M])),axis=1)
dfY0['Pop5564_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['Pop5564_Y0E'],x['Pop5564_Y0M'])),axis=1)
dfY0['LF5564_Y0E'] = dfY0.loc[:,LF5564E].sum(axis=1)
dfY0['LF5564_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LF5564M])),axis=1)
dfY0['LF5564_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF5564_Y0E'],x['LF5564_Y0M'])),axis=1)

dfY0['PopO65_Y0E'] = dfY0.loc[:,PopO65E].sum(axis=1)
dfY0['PopO65_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[PopO65M])),axis=1)
dfY0['PopO65_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['PopO65_Y0E'],x['PopO65_Y0M'])),axis=1)
dfY0['LFO65_Y0E'] = dfY0.loc[:,LFO65E].sum(axis=1)
dfY0['LFO65_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LFO65M])),axis=1)
dfY0['LFO65_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LFO65_Y0E'],x['LFO65_Y0M'])),axis=1)

dfY0['Pop2534_Y0E'] = dfY0.loc[:,Pop2534E].sum(axis=1)
dfY0['Pop2534_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[Pop2534M])),axis=1)
dfY0['Pop2534_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['Pop2534_Y0E'],x['Pop2534_Y0M'])),axis=1)
dfY0['LF2534_Y0E'] = dfY0.loc[:,LF2534E].sum(axis=1)
dfY0['LF2534_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LF2534M])),axis=1)
dfY0['LF2534_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF2534_Y0E'],x['LF2534_Y0M'])),axis=1)

dfY0['Pop3544_Y0E'] = dfY0.loc[:,Pop3544E].sum(axis=1)
dfY0['Pop3544_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[Pop3544M])),axis=1)
dfY0['Pop3544_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['Pop3544_Y0E'],x['Pop3544_Y0M'])),axis=1)
dfY0['LF3544_Y0E'] = dfY0.loc[:,LF3544E].sum(axis=1)
dfY0['LF3544_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LF3544M])),axis=1)
dfY0['LF3544_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF3544_Y0E'],x['LF3544_Y0M'])),axis=1)

dfY0['Pop4554_Y0E'] = dfY0.loc[:,Pop4554E].sum(axis=1)
dfY0['Pop4554_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[Pop4554M])),axis=1)
dfY0['Pop4554_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['Pop4554_Y0E'],x['Pop4554_Y0M'])),axis=1)
dfY0['LF4554_Y0E'] = dfY0.loc[:,LF4554E].sum(axis=1)
dfY0['LF4554_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LF4554M])),axis=1)
dfY0['LF4554_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LF4554_Y0E'],x['LF4554_Y0M'])),axis=1)

dfY0['PopO55_Y0E'] = dfY0.loc[:,PopO55E].sum(axis=1)
dfY0['PopO55_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[PopO55M])),axis=1)
dfY0['PopO55_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['PopO55_Y0E'],x['PopO55_Y0M'])),axis=1)
dfY0['LFO55_Y0E'] = dfY0.loc[:,LFO55E].sum(axis=1)
dfY0['LFO55_Y0M'] = dfY0.apply(lambda x: (calc.get_moe(x[LFO55M])),axis=1)
dfY0['LFO55_Y0C'] = dfY0.apply(lambda x: (calc.get_cv(x['LFO55_Y0E'],x['LFO55_Y0M'])),axis=1)
    
dfY0.head()

,GEO_ID,B23001_001E,B23001_001M,DP03_0002E,DP03_0002M,B23001_003E,B23001_010E,B23001_017E,B23001_089E,B23001_096E,...,Pop4554_Y0C,LF4554_Y0E,LF4554_Y0M,LF4554_Y0C,PopO55_Y0E,PopO55_Y0M,PopO55_Y0C,LFO55_Y0E,LFO55_Y0M,LFO55_Y0C
0,0500000US13155,7589.0,99.0,3908.0,649.0,302.0,113.0,194.0,343.0,111.0,...,5.273211,757.0,206.634944,16.593652,2595.0,287.975693,6.746097,815.0,267.555602,19.956783
1,0500000US13157,44688.0,238.0,28292.0,796.0,1982.0,613.0,1040.0,1439.0,444.0,...,1.506880,6378.0,288.736904,2.752022,13205.0,527.419188,2.428016,5057.0,384.137996,4.617729
2,0500000US13159,10609.0,53.0,6797.0,286.0,415.0,126.0,308.0,370.0,167.0,...,2.565498,1521.0,131.529464,5.256878,3404.0,278.621607,4.975759,1282.0,226.342661,10.732786
3,0500000US13161,10969.0,153.0,6257.0,369.0,319.0,63.0,310.0,445.0,260.0,...,2.711755,1415.0,127.781063,5.489644,3545.0,320.388826,5.494083,984.0,318.563965,19.680478
4,0500000US13163,13079.0,117.0,7263.0,437.0,596.0,301.0,249.0,514.0,267.0,...,4.872519,1805.0,147.851277,4.979457,4488.0,298.288451,4.040338,1226.0,199.040197,9.869256


In [12]:
dfY0 = dfY0.drop(var_data[1:],axis=1)
dfY0.head()

,GEO_ID,Pop_Y0E,Pop_Y0M,Pop_Y0C,LF_Y0E,LF_Y0M,LF_Y0C,Pop1624_Y0E,Pop1624_Y0M,Pop1624_Y0C,...,Pop4554_Y0C,LF4554_Y0E,LF4554_Y0M,LF4554_Y0C,PopO55_Y0E,PopO55_Y0M,PopO55_Y0C,LFO55_Y0E,LFO55_Y0M,LFO55_Y0C
0,0500000US13155,7589.0,99.0,0.793021,3908.0,649.0,10.095416,1208.0,278.643141,14.022179,...,5.273211,757.0,206.634944,16.593652,2595.0,287.975693,6.746097,815.0,267.555602,19.956783
1,0500000US13157,44688.0,238.0,0.323758,28292.0,796.0,1.710344,6522.0,442.694025,4.126264,...,1.506880,6378.0,288.736904,2.752022,13205.0,527.419188,2.428016,5057.0,384.137996,4.617729
2,0500000US13159,10609.0,53.0,0.303694,6797.0,286.0,2.557896,1615.0,224.644163,8.455839,...,2.565498,1521.0,131.529464,5.256878,3404.0,278.621607,4.975759,1282.0,226.342661,10.732786
3,0500000US13161,10969.0,153.0,0.847927,6257.0,369.0,3.585043,1838.0,308.413683,10.200518,...,2.711755,1415.0,127.781063,5.489644,3545.0,320.388826,5.494083,984.0,318.563965,19.680478
4,0500000US13163,13079.0,117.0,0.543808,7263.0,437.0,3.657628,2151.0,236.689670,6.689182,...,4.872519,1805.0,147.851277,4.979457,4488.0,298.288451,4.040338,1226.0,199.040197,9.869256


dfY0.to_excel('lf10_county_FullUS.xlsx')

## Calculate the change between the two periods
### Change between 2006-2010 5-YR Avg and 2014-2018 5-YR Avg

In [13]:
#Merge the two dataframes using the stco identifier
lfY0Y1 = pd.merge(dfY0,dfY1,how='left',on='GEO_ID')
lfY0Y1.head()

,GEO_ID,Pop_Y0E,Pop_Y0M,Pop_Y0C,LF_Y0E,LF_Y0M,LF_Y0C,Pop1624_Y0E,Pop1624_Y0M,Pop1624_Y0C,...,Pop4554_Y1C,LF4554_Y1E,LF4554_Y1M,LF4554_Y1C,PopO55_Y1E,PopO55_Y1M,PopO55_Y1C,LFO55_Y1E,LFO55_Y1M,LFO55_Y1C
0,0500000US13155,7589.0,99.0,0.793021,3908.0,649.0,10.095416,1208.0,278.643141,14.022179,...,8.162728,982.0,168.359734,10.422235,2817.0,253.148178,5.462885,983.0,192.730382,11.918751
1,0500000US13157,44688.0,238.0,0.323758,28292.0,796.0,1.710344,6522.0,442.694025,4.126264,...,1.968134,7657.0,371.587136,2.950096,17854.0,632.026898,2.151960,6266.0,477.381399,4.631367
2,0500000US13159,10609.0,53.0,0.303694,6797.0,286.0,2.557896,1615.0,224.644163,8.455839,...,3.293115,1452.0,119.753914,5.013687,4488.0,310.088697,4.200173,1435.0,217.073720,9.195798
3,0500000US13161,10969.0,153.0,0.847927,6257.0,369.0,3.585043,1838.0,308.413683,10.200518,...,4.985288,1027.0,239.793661,14.193887,4071.0,344.981159,5.151437,1081.0,234.384300,13.180653
4,0500000US13163,13079.0,117.0,0.543808,7263.0,437.0,3.657628,2151.0,236.689670,6.689182,...,1.069187,1323.0,147.989865,6.799958,4936.0,307.507723,3.787172,1595.0,225.674544,8.601139


In [14]:
#Calculate change columns (net change, change moe, and change cv) and add to the table

lfY0Y1['Pop_Y0Y1E'] = lfY0Y1.Pop_Y1E - lfY0Y1.Pop_Y0E 
lfY0Y1['Pop_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['Pop_Y0M'],x['Pop_Y1M']])),axis=1)
lfY0Y1['Pop_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['Pop_Y0Y1E'],x['Pop_Y0Y1M'])),axis=1)
lfY0Y1['LF_Y0Y1E'] = lfY0Y1.LF_Y1E - lfY0Y1.LF_Y0E
lfY0Y1['LF_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LF_Y0M'],x['LF_Y1M']])),axis=1)
lfY0Y1['LF_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LF_Y0Y1E'],x['LF_Y0Y1M'])),axis=1)

lfY0Y1['Pop1624_Y0Y1E'] = lfY0Y1.Pop1624_Y1E - lfY0Y1.Pop1624_Y0E
lfY0Y1['Pop1624_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['Pop1624_Y0M'],x['Pop1624_Y1M']])),axis=1)
lfY0Y1['Pop1624_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['Pop1624_Y0Y1E'],x['Pop1624_Y0Y1M'])),axis=1)
lfY0Y1['LF1624_Y0Y1E'] = lfY0Y1.LF1624_Y1E - lfY0Y1.LF1624_Y0E
lfY0Y1['LF1624_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LF1624_Y0M'],x['LF1624_Y1M']])),axis=1)
lfY0Y1['LF1624_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LF1624_Y0Y1E'],x['LF1624_Y0Y1M'])),axis=1)

lfY0Y1['Pop2554_Y0Y1E'] = lfY0Y1.Pop2554_Y1E - lfY0Y1.Pop2554_Y0E
lfY0Y1['Pop2554_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['Pop2554_Y0M'],x['Pop2554_Y1M']])),axis=1)
lfY0Y1['Pop2554_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['Pop2554_Y0Y1E'],x['Pop2554_Y0Y1M'])),axis=1)
lfY0Y1['LF2554_Y0Y1E'] = lfY0Y1.LF2554_Y1E - lfY0Y1.LF2554_Y0E
lfY0Y1['LF2554_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LF2554_Y0M'],x['LF2554_Y1M']])),axis=1)
lfY0Y1['LF2554_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LF2554_Y0Y1E'],x['LF2554_Y0Y1M'])),axis=1)

lfY0Y1['Pop5564_Y0Y1E'] = lfY0Y1.Pop5564_Y1E - lfY0Y1.Pop5564_Y0E
lfY0Y1['Pop5564_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['Pop5564_Y0M'],x['Pop5564_Y1M']])),axis=1)
lfY0Y1['Pop5564_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['Pop5564_Y0Y1E'],x['Pop5564_Y0Y1M'])),axis=1)
lfY0Y1['LF5564_Y0Y1E'] = lfY0Y1.LF5564_Y1E - lfY0Y1.LF5564_Y0E
lfY0Y1['LF5564_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LF5564_Y0M'],x['LF5564_Y1M']])),axis=1)
lfY0Y1['LF5564_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LF5564_Y0Y1E'],x['LF5564_Y0Y1M'])),axis=1)

lfY0Y1['PopO65_Y0Y1E'] = lfY0Y1.PopO65_Y1E - lfY0Y1.PopO65_Y0E
lfY0Y1['PopO65_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['PopO65_Y0M'],x['PopO65_Y1M']])),axis=1)
lfY0Y1['PopO65_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['PopO65_Y0Y1E'],x['PopO65_Y0Y1M'])),axis=1)
lfY0Y1['LFO65_Y0Y1E'] = lfY0Y1.LFO65_Y1E - lfY0Y1.LFO65_Y0E
lfY0Y1['LFO65_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LFO65_Y0M'],x['LFO65_Y1M']])),axis=1)
lfY0Y1['LFO65_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LFO65_Y0Y1E'],x['LFO65_Y0Y1M'])),axis=1)

lfY0Y1['Pop2534_Y0Y1E'] = lfY0Y1.Pop2534_Y1E - lfY0Y1.Pop2534_Y0E
lfY0Y1['Pop2534_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['Pop2534_Y0M'],x['Pop2534_Y1M']])),axis=1)
lfY0Y1['Pop2534_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['Pop2534_Y0Y1E'],x['Pop2534_Y0Y1M'])),axis=1)
lfY0Y1['LF2534_Y0Y1E'] = lfY0Y1.LF2534_Y1E - lfY0Y1.LF2534_Y0E
lfY0Y1['LF2534_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LF2534_Y0M'],x['LF2534_Y1M']])),axis=1)
lfY0Y1['LF2534_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LF2534_Y0Y1E'],x['LF2534_Y0Y1M'])),axis=1)

lfY0Y1['Pop3544_Y0Y1E'] = lfY0Y1.Pop3544_Y1E - lfY0Y1.Pop3544_Y0E
lfY0Y1['Pop3544_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['Pop3544_Y0M'],x['Pop3544_Y1M']])),axis=1)
lfY0Y1['Pop3544_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['Pop3544_Y0Y1E'],x['Pop3544_Y0Y1M'])),axis=1)
lfY0Y1['LF3544_Y0Y1E'] = lfY0Y1.LF3544_Y1E - lfY0Y1.LF3544_Y0E
lfY0Y1['LF3544_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LF3544_Y0M'],x['LF3544_Y1M']])),axis=1)
lfY0Y1['LF3544_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LF3544_Y0Y1E'],x['LF3544_Y0Y1M'])),axis=1)

lfY0Y1['Pop4554_Y0Y1E'] = lfY0Y1.Pop4554_Y1E - lfY0Y1.Pop4554_Y0E
lfY0Y1['Pop4554_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['Pop4554_Y0M'],x['Pop4554_Y1M']])),axis=1)
lfY0Y1['Pop4554_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['Pop4554_Y0Y1E'],x['Pop4554_Y0Y1M'])),axis=1)
lfY0Y1['LF4554_Y0Y1E'] = lfY0Y1.LF4554_Y1E - lfY0Y1.LF4554_Y0E
lfY0Y1['LF4554_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LF4554_Y0M'],x['LF4554_Y1M']])),axis=1)
lfY0Y1['LF4554_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LF4554_Y0Y1E'],x['LF4554_Y0Y1M'])),axis=1)

lfY0Y1['PopO55_Y0Y1E'] = lfY0Y1.PopO55_Y1E - lfY0Y1.PopO55_Y0E
lfY0Y1['PopO55_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['PopO55_Y0M'],x['PopO55_Y1M']])),axis=1)
lfY0Y1['PopO55_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['PopO55_Y0Y1E'],x['PopO55_Y0Y1M'])),axis=1)
lfY0Y1['LFO55_Y0Y1E'] = lfY0Y1.LFO55_Y1E - lfY0Y1.LFO55_Y0E
lfY0Y1['LFO55_Y0Y1M'] = lfY0Y1.apply(lambda x: (calc.get_moe([x['LFO55_Y0M'],x['LFO55_Y1M']])),axis=1)
lfY0Y1['LFO55_Y0Y1C'] = lfY0Y1.apply(lambda x: (calc.get_cv(x['LFO55_Y0Y1E'],x['LFO55_Y0Y1M'])),axis=1)

lfY0Y1.head()

,GEO_ID,Pop_Y0E,Pop_Y0M,Pop_Y0C,LF_Y0E,LF_Y0M,LF_Y0C,Pop1624_Y0E,Pop1624_Y0M,Pop1624_Y0C,...,Pop4554_Y0Y1C,LF4554_Y0Y1E,LF4554_Y0Y1M,LF4554_Y0Y1C,PopO55_Y0Y1E,PopO55_Y0Y1M,PopO55_Y0Y1C,LFO55_Y0Y1E,LFO55_Y0Y1M,LFO55_Y0Y1C
0,0500000US13155,7589.0,99.0,0.793021,3908.0,649.0,10.095416,1208.0,278.643141,14.022179,...,102.384667,225.0,266.538928,72.013219,222.0,383.424047,104.993030,168.0,329.743840,119.316775
1,0500000US13157,44688.0,238.0,0.323758,28292.0,796.0,1.710344,6522.0,442.694025,4.126264,...,16.248286,1279.0,470.580493,22.366471,4649.0,823.182240,10.763922,1209.0,612.743829,30.809648
2,0500000US13159,10609.0,53.0,0.303694,6797.0,286.0,2.557896,1615.0,224.644163,8.455839,...,88.020808,-69.0,177.879172,156.714834,1084.0,416.875281,23.378194,153.0,313.611224,124.604654
3,0500000US13161,10969.0,153.0,0.847927,6257.0,369.0,3.585043,1838.0,308.413683,10.200518,...,45.896998,-388.0,271.714924,42.571197,526.0,470.808878,54.411788,97.0,395.498420,247.860383
4,0500000US13163,13079.0,117.0,0.543808,7263.0,437.0,3.657628,2151.0,236.689670,6.689182,...,36.785961,-482.0,209.191300,26.383395,448.0,428.412185,58.132352,369.0,300.908624,49.572676


In [16]:
lfY0Y1.to_excel(f'../../output/Labor Force/LF_UScounty_{date}.xlsx')

# Grab NYC Metro Region counties only

In [17]:
df_31cr = lfY0Y1.loc[lfY0Y1['GEO_ID'].isin(stco)].copy()
df_31cr.shape

(32, 163)

In [18]:
for column_name in df_31cr.columns:
    df_31cr.rename(columns={column_name:column_name.replace('Y0',y0[2:]).replace('Y1',y1[2:])},inplace=True)

In [19]:
df_31cr.head()

,GEO_ID,Pop_10E,Pop_10M,Pop_10C,LF_10E,LF_10M,LF_10C,Pop1624_10E,Pop1624_10M,Pop1624_10C,...,Pop4554_1019C,LF4554_1019E,LF4554_1019M,LF4554_1019C,PopO55_1019E,PopO55_1019M,PopO55_1019C,LFO55_1019E,LFO55_1019M,LFO55_1019C
438,0500000US09001,703812.0,616.0,0.053206,475682.0,2304.0,0.294442,96465.0,1372.730126,0.865066,...,1.988251,-2999.0,1228.101380,24.893838,54415.0,2866.262375,3.202074,36218.0,2610.114174,4.380958
440,0500000US09005,153468.0,245.0,0.097047,107531.0,846.0,0.478267,18671.0,550.672316,1.792915,...,2.059770,-5433.0,592.312418,6.627431,13340.0,1415.832617,6.451938,6975.0,1217.678529,10.612618
442,0500000US09009,686161.0,694.0,0.061485,465571.0,2465.0,0.321859,108450.0,1740.966111,0.975876,...,0.823304,-12082.0,1292.009675,6.500714,43557.0,2918.965056,4.073850,23136.0,2504.595975,6.580873
1900,0500000US34003,717394.0,589.0,0.049910,475551.0,2700.0,0.345144,91762.0,1312.998096,0.869832,...,0.907965,-5038.0,1368.297482,16.510357,45484.0,3170.821345,4.237866,31276.0,2810.568448,5.462822
1905,0500000US34013,607411.0,556.0,0.055645,399973.0,2710.0,0.411882,96204.0,1417.450528,0.895672,...,18.832842,759.0,1408.250333,112.790412,32422.0,2778.034737,5.208731,18445.0,2364.223128,7.791909


In [22]:
df_31cr.to_excel(f'../../output/Labor Force/LF_31county_{date}.xlsx')

## Subregion Calc

In [23]:
df_31cr['stco'] = df_31cr['GEO_ID'].apply(lambda x: x[-5:])
df_31cr.head()

,GEO_ID,Pop_10E,Pop_10M,Pop_10C,LF_10E,LF_10M,LF_10C,Pop1624_10E,Pop1624_10M,Pop1624_10C,...,LF4554_1019E,LF4554_1019M,LF4554_1019C,PopO55_1019E,PopO55_1019M,PopO55_1019C,LFO55_1019E,LFO55_1019M,LFO55_1019C,stco
438,0500000US09001,703812.0,616.0,0.053206,475682.0,2304.0,0.294442,96465.0,1372.730126,0.865066,...,-2999.0,1228.101380,24.893838,54415.0,2866.262375,3.202074,36218.0,2610.114174,4.380958,09001
440,0500000US09005,153468.0,245.0,0.097047,107531.0,846.0,0.478267,18671.0,550.672316,1.792915,...,-5433.0,592.312418,6.627431,13340.0,1415.832617,6.451938,6975.0,1217.678529,10.612618,09005
442,0500000US09009,686161.0,694.0,0.061485,465571.0,2465.0,0.321859,108450.0,1740.966111,0.975876,...,-12082.0,1292.009675,6.500714,43557.0,2918.965056,4.073850,23136.0,2504.595975,6.580873,09009
1900,0500000US34003,717394.0,589.0,0.049910,475551.0,2700.0,0.345144,91762.0,1312.998096,0.869832,...,-5038.0,1368.297482,16.510357,45484.0,3170.821345,4.237866,31276.0,2810.568448,5.462822,34003
1905,0500000US34013,607411.0,556.0,0.055645,399973.0,2710.0,0.411882,96204.0,1417.450528,0.895672,...,759.0,1408.250333,112.790412,32422.0,2778.034737,5.208731,18445.0,2364.223128,7.791909,34013


In [26]:
geo_xwalk = pd.read_csv('../data/geo/coxsub.csv')
geo_xwalk['stco'] = geo_xwalk['stco'].apply(lambda x: '{0:0>5}'.format(x))

In [27]:
df_subreg = geo_xwalk.merge(df_31cr,on='stco')
df_subreg = df_subreg.drop(columns=['stco','st','co','stco_int','subreg2','reg','stco_lbl','co_lbl','GEO_ID'])
df_subreg.head()

,subreg1,Pop_10E,Pop_10M,Pop_10C,LF_10E,LF_10M,LF_10C,Pop1624_10E,Pop1624_10M,Pop1624_10C,...,Pop4554_1019C,LF4554_1019E,LF4554_1019M,LF4554_1019C,PopO55_1019E,PopO55_1019M,PopO55_1019C,LFO55_1019E,LFO55_1019M,LFO55_1019C
0,CT,703812.0,616.0,0.053206,475682.0,2304.0,0.294442,96465.0,1372.730126,0.865066,...,1.988251,-2999.0,1228.101380,24.893838,54415.0,2866.262375,3.202074,36218.0,2610.114174,4.380958
1,CT,153468.0,245.0,0.097047,107531.0,846.0,0.478267,18671.0,550.672316,1.792915,...,2.059770,-5433.0,592.312418,6.627431,13340.0,1415.832617,6.451938,6975.0,1217.678529,10.612618
2,CT,686161.0,694.0,0.061485,465571.0,2465.0,0.321859,108450.0,1740.966111,0.975876,...,0.823304,-12082.0,1292.009675,6.500714,43557.0,2918.965056,4.073850,23136.0,2504.595975,6.580873
3,NJ In,717394.0,589.0,0.049910,475551.0,2700.0,0.345144,91762.0,1312.998096,0.869832,...,0.907965,-5038.0,1368.297482,16.510357,45484.0,3170.821345,4.237866,31276.0,2810.568448,5.462822
4,NJ In,607411.0,556.0,0.055645,399973.0,2710.0,0.411882,96204.0,1417.450528,0.895672,...,18.832842,759.0,1408.250333,112.790412,32422.0,2778.034737,5.208731,18445.0,2364.223128,7.791909


In [28]:
df_subreg = geo.calculate_sumgeo(df_subreg,'subreg1')
df_subreg

,subreg1,Pop_19E,Pop_19M,Pop_19C,Pop4554_10E,Pop4554_10M,Pop4554_10C,Pop5564_1019E,Pop5564_1019M,Pop5564_1019C,...,Pop2534_1019C,LF5564_19E,LF5564_19M,LF5564_19C,Pop_10E,Pop_10M,Pop_10C,LF2554_10E,LF2554_10M,LF2554_10C
0,CT,1613048.0,981.396964,0.036986,309374.0,194.118520,0.038143,52049.0,3567.674733,4.166841,...,1.578034,207935.0,2480.273775,0.725114,1543441.0,959.748405,0.037801,694888.0,2178.426955,0.190573
1,NJ In,4097756.0,1437.426868,0.021324,748920.0,358.593084,0.029107,118185.0,5747.660568,2.956398,...,1.000524,479451.0,3941.175332,0.499707,3901209.0,1429.121758,0.022269,1826521.0,3902.402209,0.129880
2,NJ Out,1582480.0,939.805831,0.036102,311369.0,263.150147,0.051376,49490.0,3574.087576,4.390175,...,2.939992,205321.0,2496.152640,0.739047,1539526.0,900.309947,0.035550,666934.0,2183.573905,0.199030
3,NYC,6846849.0,1734.569399,0.015401,1092708.0,372.116917,0.020702,141492.0,7792.325648,3.347876,...,0.259245,640798.0,5265.639752,0.499533,6510606.0,1977.428128,0.018463,2926143.0,5832.164950,0.121163
4,Mid Hud,751600.0,609.962294,0.049334,147984.0,171.508017,0.070454,21446.0,2467.102754,6.993185,...,3.291967,87093.0,1619.041383,1.130079,729040.0,651.357045,0.054313,313744.0,1645.920107,0.318909
5,LI,2306035.0,1076.144972,0.028369,451396.0,144.696234,0.019486,70955.0,4316.478773,3.698117,...,0.589348,294179.0,2931.933492,0.605866,2218922.0,1076.551903,0.029494,986860.0,2842.482366,0.175096
6,Low Hud,1105592.0,726.095035,0.039924,207863.0,172.820716,0.050542,28120.0,2924.723577,6.322715,...,1.903137,135831.0,1996.496932,0.893519,1046372.0,758.116746,0.044044,457594.0,2043.734083,0.271505


In [29]:
df_subreg.to_excel(f'../../output/Labor Force/LF_subregion_{date}.xlsx')

## Region Calc

In [30]:
df_reg = geo_xwalk.merge(df_31cr,on='stco')
df_reg = df_reg.drop(columns=['stco','st','co','stco_int','subreg2','subreg1','stco_lbl','co_lbl','GEO_ID'])
df_reg.head()

,reg,Pop_10E,Pop_10M,Pop_10C,LF_10E,LF_10M,LF_10C,Pop1624_10E,Pop1624_10M,Pop1624_10C,...,Pop4554_1019C,LF4554_1019E,LF4554_1019M,LF4554_1019C,PopO55_1019E,PopO55_1019M,PopO55_1019C,LFO55_1019E,LFO55_1019M,LFO55_1019C
0,31CR,703812.0,616.0,0.053206,475682.0,2304.0,0.294442,96465.0,1372.730126,0.865066,...,1.988251,-2999.0,1228.101380,24.893838,54415.0,2866.262375,3.202074,36218.0,2610.114174,4.380958
1,31CR,153468.0,245.0,0.097047,107531.0,846.0,0.478267,18671.0,550.672316,1.792915,...,2.059770,-5433.0,592.312418,6.627431,13340.0,1415.832617,6.451938,6975.0,1217.678529,10.612618
2,31CR,686161.0,694.0,0.061485,465571.0,2465.0,0.321859,108450.0,1740.966111,0.975876,...,0.823304,-12082.0,1292.009675,6.500714,43557.0,2918.965056,4.073850,23136.0,2504.595975,6.580873
3,31CR,717394.0,589.0,0.049910,475551.0,2700.0,0.345144,91762.0,1312.998096,0.869832,...,0.907965,-5038.0,1368.297482,16.510357,45484.0,3170.821345,4.237866,31276.0,2810.568448,5.462822
4,31CR,607411.0,556.0,0.055645,399973.0,2710.0,0.411882,96204.0,1417.450528,0.895672,...,18.832842,759.0,1408.250333,112.790412,32422.0,2778.034737,5.208731,18445.0,2364.223128,7.791909


In [31]:
df_reg = geo.calculate_sumgeo(df_reg,'reg')
df_reg

,reg,Pop_19E,Pop_19M,Pop_19C,Pop4554_10E,Pop4554_10M,Pop4554_10C,Pop5564_1019E,Pop5564_1019M,Pop5564_1019C,...,Pop2534_1019C,LF5564_19E,LF5564_19M,LF5564_19C,Pop_10E,Pop_10M,Pop_10C,LF2554_10E,LF2554_10M,LF2554_10C
0,31CR,18303360.0,2996.440889,0.009952,3269614.0,673.950295,0.01253,481737.0,12350.341655,1.558487,...,0.291574,2050608.0,8417.000059,0.249522,17489116.0,3137.23716,0.010905,7872684.0,8586.153854,0.066299


In [32]:
df_reg.to_excel(f'../../output/Labor Force/LF_region_{date}.xlsx')